In [57]:
%matplotlib inline

import numpy as np 
import pandas as pd 
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology

INPUT_FOLDER = 'input/sample_images/'


In [66]:
def load_slices(path):
    all_slices = []
    
    patients = os.listdir(path)
    patients.sort()
    patients = patients[1:]
    patients_IDs = []
    for patient in patients:
        slices = []
        patients_IDs.append(patient)
        for s in os.listdir(path + patient): 
            if s[-4:] == '.dcm':
                slices.append(dicom.read_file(path + patient + '/' + s))   
                slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    
        slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
        try:
            slice_thickness = np.abs(slices[10].ImagePositionPatient[2] - slices[0].ImagePositionPatient[2])/10
        except:
            slice_thickness = np.abs(slices[10].SliceLocation - slices[0].SliceLocation)/10
        for s in slices:
            s.SliceThickness = slice_thickness
        all_slices.append(slices)
        
    return all_slices, patients_IDs

In [67]:
#all_slices is a nested array, one object in all_slices is slices of one patient
#In this way, all_slices stores slices for all patients
all_slices, patients_IDs = load_slices(INPUT_FOLDER)

00cba091fa4ad62cc3200a657aeb957e
0a099f2549429d29b32f349e95fb2244
0a0c32c9e08cc2ea76a71649de56be6d
0a38e7597ca26f9374f8ea2770ba870d
0acbebb8d463b4b9ca88cf38431aac69
0b20184e0cd497028bdd155d9fb42dc9
0bd0e3056cbf23a1cb7f0f0b18446068
0c0de3749d4fe175b7a5098b060982a1
0c37613214faddf8701ca41e6d43f56e
0c59313f52304e25d5a7dcf9877633b1
0c60f4b87afcb3e2dfa65abbbf3ef2f9
0c98fcb55e3f36d0c2b6507f62f4c5f1
0c9d8314f9c69840e25febabb1229fa4
0ca943d821204ceb089510f836a367fd
0d06d764d3c07572074d468b4cff954f
0d19f1c627df49eb223771c28548350e
0d2fcf787026fece4e57be167d079383
0d941a3ad6c889ac451caf89c46cb92a
0ddeb08e9c97227853422bd71a2a695e
0de72529c30fe642bc60dcb75c87f6bd


In [16]:
def convert_to_HU(slices):
    #images is 3d array, it stores 2d pixel_arrays from each slice belonging to a patient
    images = np.stack([s.pixel_array for s in slices])
    images = images.astype(np.int16)

    # Set outside-of-scan pixels to 0
    images[images == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):      
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            images[slice_number] = slope * images[slice_number].astype(np.float64)
            images[slice_number] = images[slice_number].astype(np.int16)
            
        images[slice_number] += np.int16(intercept)
    
    return np.array(images, dtype=np.int16)

In [26]:
def resample(images, slices, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([slices[0].SliceThickness] + slices[0].PixelSpacing, dtype=np.float32)
    resize_factor = spacing / new_spacing
    new_real_shape = images.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / images.shape
    new_spacing = spacing / real_resize_factor
    
    images = scipy.ndimage.interpolation.zoom(images, real_resize_factor, mode='nearest')
    
    return images, new_spacing

In [28]:
all_resampled = []
all_spacing = []
for slices in all_slices:
    pixels_HU = convert_to_HU(slices)
    pixels_resampled, spacing  = resample(pixels_HU, slices, [1,1,1]) 
    all_resampled.append(pixels_resampled)
    all_spacing.append(spacing)
    print(pixels_HU.shape)
    print(pixels_resampled.shape)

(134, 512, 512)
(335, 306, 306)
(128, 512, 512)
(320, 347, 347)
(133, 512, 512)
(332, 340, 340)
(110, 512, 512)
(275, 320, 320)
(203, 512, 512)
(365, 279, 279)
(196, 512, 512)
(353, 360, 360)
(280, 512, 512)
(350, 340, 340)
(123, 512, 512)
(308, 355, 355)
(164, 512, 512)
(328, 360, 360)
(244, 512, 512)
(305, 390, 390)
(136, 512, 512)
(272, 330, 330)
(180, 512, 512)
(360, 350, 350)
(221, 512, 512)
(398, 309, 309)
(147, 512, 512)
(294, 300, 300)
(435, 512, 512)
(304, 424, 424)
(183, 512, 512)
(366, 370, 370)
(126, 512, 512)
(315, 310, 310)
(177, 512, 512)
(354, 259, 259)
(171, 512, 512)
(342, 392, 392)
(113, 512, 512)
(282, 308, 308)


In [30]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(pixels, fill_lung_structures=True):
    # 0 is treated as background, which is non-ROI
    # Use -320 as a threshold based on wikipedia and the histogram we generated above
    binary_image = np.array(pixels > -320, dtype=np.int8)+1
    
    #label connnected regions with an integer
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
     
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: 
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: 
        binary_image[labels != l_max] = 0
 
    return binary_image

In [31]:
MIN_BOUND = -1000.0
MAX_BOUND = 400.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

PIXEL_MEAN = 0.25

def zero_center(image):
    image = image - PIXEL_MEAN
    return image

In [ ]:
from skimage.morphology import disk,ball
from skimage import morphology

all_images = []
for pixel_resampled, patient_ID in zip(all_resampled, patients_IDs):
    segmented_lungs_fill = segment_lung_mask(pixel_resampled, True)
    eroded = morphology.erosion(segmented_lungs_fill,ball(1))
    dilation = morphology.dilation(eroded,ball(3))
    images = np.multiply(dilation, pixel_resampled)

    images = normalize(images)
    images = zero_center(images)
    np.save(patient_ID + ".npy", images)
    all_images.append(images)